## **Install & Imports**

In [1]:
!pip install unsloth
!pip install peft==0.18.0
!pip install evaluate rouge-score sacrebleu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.6/66.6 kB 2.8 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of torchvision to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 381.1/381.1 kB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 34.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 506.8/506.8 kB 32.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 423.1/423.1 kB 33.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.7/295.7 kB 21.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.9/122.9 MB 15.7 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 899.7/899.7 MB 1.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 170.5/170.5 MB 2.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 594.3/594.3 MB 3.0 MB/s eta 0:00:00:00:01

## **Global Config**

In [5]:
MODEL_NAME = "unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit"
MAX_SEQ_LENGTH = 4096   # ❗ do not reduce
LOAD_IN_4BIT = True
DTYPE = None            # auto (fp16 on T4)

## ***Check specific version that's compatible with Unsloth***

In [6]:
from unsloth import FastLanguageModel
import trl, peft, transformers, bitsandbytes, torch, xformers

print("Unsloth OK ✅")
print("trl:", trl.__version__)
print("peft:", peft.__version__)
print("transformers:", transformers.__version__)
print("xformers:", xformers.__version__)
print("bnb:", bitsandbytes.__version__)
print("torch:", torch.__version__)
print("CUDA:", torch.cuda.is_available())
print("GPU:", torch.cuda.get_device_name())
print(torch.cuda.device_count())

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


2026-01-08 10:00:23.232792: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1767866423.447540      55 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1767866423.506978      55 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1767866424.013366      55 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1767866424.013397      55 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1767866424.013400      55 computation_placer.cc:177] computation placer alr

🦥 Unsloth Zoo will now patch everything to make training faster!
Unsloth OK ✅
trl: 0.24.0
peft: 0.18.0
transformers: 4.57.1
xformers: 0.0.33.post2
bnb: 0.49.0
torch: 2.9.1+cu128
CUDA: True
GPU: Tesla T4
2


## **Load Model + Tokenizer**

In [7]:
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = MODEL_NAME,
    max_seq_length = MAX_SEQ_LENGTH,
    dtype = DTYPE,
    load_in_4bit = LOAD_IN_4BIT,
    device_map = "auto" 
)

tokenizer.pad_token = tokenizer.eos_token

print("Device mapping:", model.hf_device_map)
print("Model is on:", next(model.parameters()).device)

==((====))==  Unsloth 2026.1.2: Fast Llama patching. Transformers: 4.57.1.
   \\   /|    Tesla T4. Num GPUs = 2. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.1+cu128. CUDA: 7.5. CUDA Toolkit: 12.8. Triton: 3.5.1
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.33.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.70G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/454 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

Device mapping: {'model.embed_tokens': 0, 'model.layers.0': 0, 'model.layers.1': 0, 'model.layers.2': 0, 'model.layers.3': 0, 'model.layers.4': 0, 'model.layers.5': 0, 'model.layers.6': 0, 'model.layers.7': 0, 'model.layers.8': 0, 'model.layers.9': 1, 'model.layers.10': 1, 'model.layers.11': 1, 'model.layers.12': 1, 'model.layers.13': 1, 'model.layers.14': 1, 'model.layers.15': 1, 'model.layers.16': 1, 'model.layers.17': 1, 'model.layers.18': 1, 'model.layers.19': 1, 'model.layers.20': 1, 'model.layers.21': 1, 'model.layers.22': 1, 'model.layers.23': 1, 'model.layers.24': 1, 'model.layers.25': 1, 'model.layers.26': 1, 'model.layers.27': 1, 'model.layers.28': 1, 'model.layers.29': 1, 'model.layers.30': 1, 'model.layers.31': 1, 'model.norm': 1, 'model.rotary_emb': 1, 'lm_head': 1}
Model is on: cuda:0


## **Apply PEFT (LoRA via Unsloth)**

In [8]:
model = FastLanguageModel.get_peft_model(
        model,
        r=16,
        target_modules=["q_proj", "k_proj", "v_proj", "o_proj",
                        "gate_proj", "up_proj", "down_proj"],
        lora_alpha=16,
        lora_dropout=0,  # Optimized when set to 0
        bias="none",     # Optimized when set to "none"
        use_gradient_checkpointing="unsloth",  # "unsloth" for lower VRAM usage
        random_state=3407,
        max_seq_length=MAX_SEQ_LENGTH,
        use_rslora=False,  # Rank-stabilized LoRA (set to True if needed)
        loftq_config=None,  # LoftQ config (set if using LoftQ)
    )

Unsloth 2026.1.2 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


## **Dataset Prompt Formatter**

In [10]:
EOS = tokenizer.eos_token

PROMPT = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""


In [11]:
def format_examples(batch):
    texts = []
    for inst, q, ans in zip(
        batch["Question-Title"],
        batch["Questions"],
        batch["Answers"]
    ):
        texts.append(PROMPT.format(inst, q, ans) + EOS)
    return {"text": texts}

## **Load Dataset**

In [18]:
from datasets import load_dataset

dataset = load_dataset(
    "csv", 
    data_files="/kaggle/input/bengali-empathetic-conversations-corpus/BengaliEmpatheticConversationsCorpus .csv", 
    split="train"
)



## **Preprocess (❗ NO TOKENIZATION)**

In [19]:
dataset = dataset.map(
    format_examples,
    batched=True,
    remove_columns=dataset.column_names
)


Map:   0%|          | 0/38233 [00:00<?, ? examples/s]

## **Trainer Setup**

In [21]:
from trl import SFTTrainer
from transformers import TrainingArguments

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = MAX_SEQ_LENGTH,
    packing = False,  # ❗ required (no seq shortening)
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        max_steps = 60,
        learning_rate = 2e-4,
        fp16 = True,
        logging_steps = 1,
        optim = "paged_adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "./llama-bengali-empathy",
        report_to = "none",
    ),
)

Unsloth: Tokenizing ["text"] (num_proc=8):   0%|          | 0/38233 [00:00<?, ? examples/s]

## **Train**

In [22]:
trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 2
   \\   /|    Num examples = 38,233 | Num Epochs = 1 | Total steps = 60
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 41,943,040 of 8,072,204,288 (0.52% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
1,1.374100
2,1.420100
3,1.375500
4,1.294100
5,1.137500
6,0.950100
7,0.909600
8,0.861700
9,0.801800
10,0.733200


TrainOutput(global_step=60, training_loss=0.7286780685186386, metrics={'train_runtime': 516.0082, 'train_samples_per_second': 0.93, 'train_steps_per_second': 0.116, 'total_flos': 6562152412938240.0, 'train_loss': 0.7286780685186386, 'epoch': 0.012554271067636135})

## **Save PEFT Model**

In [23]:
model.save_pretrained("llama-3.1-8B-bangla-empathy")
tokenizer.save_pretrained("llama-3.1-8B-bangla-empathy")

('llama-3.1-8B-bangla-empathy/tokenizer_config.json',
 'llama-3.1-8B-bangla-empathy/special_tokens_map.json',
 'llama-3.1-8B-bangla-empathy/chat_template.jinja',
 'llama-3.1-8B-bangla-empathy/tokenizer.json')

## **Inference Test**

In [24]:
FastLanguageModel.for_inference(model)

prompt = """### Instruction:
Show empathy in Bengali

### Input:
আমি আজ খুব হতাশ

### Response:
"""

inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
outputs = model.generate(**inputs, max_new_tokens=120)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))


### Instruction:
Show empathy in Bengali

### Input:
আমি আজ খুব হতাশ

### Response:
ওহ না, আপনি খুব কষ্ট পেলেন না তো?


## **Prepare Evaluation Dataset**

In [33]:
import evaluate
import math

In [34]:
raw_eval_dataset = load_dataset(
    "csv", 
    data_files="/kaggle/input/bengali-empathetic-conversations-corpus/BengaliEmpatheticConversationsCorpus .csv", 
    split="train[:200]"   # small eval subset
)

## **Generate Predictions**

In [35]:
FastLanguageModel.for_inference(model)

def generate_response(instruction, question):
    prompt = f"""### Instruction:
{instruction}

### Input:
{question}

### Response:
"""
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    outputs = model.generate(
        **inputs,
        max_new_tokens=150,
        do_sample=True,
        temperature=0.7,
        top_p=0.9,
    )
    return tokenizer.decode(outputs[0], skip_special_tokens=True)


In [36]:
preds, refs = [], []

for row in raw_eval_dataset:
    pred = generate_response(row["Question-Title"], row["Questions"])
    preds.append(pred)
    refs.append([row["Answers"]])  # list of references

## **BLEU + ROUGE Evaluation**

In [37]:
bleu = evaluate.load("bleu")
rouge = evaluate.load("rouge")

bleu_score = bleu.compute(predictions=preds, references=refs)
rouge_score = rouge.compute(predictions=preds, references=[r[0] for r in refs])

print("BLEU:", bleu_score)
print("ROUGE:", rouge_score)

BLEU: {'bleu': 0.007669002891775697, 'precisions': [0.22454068241469816, 0.025684024713150927, 0.004096170970614425, 0.0008984725965858042], 'brevity_penalty': 0.6353728372687988, 'length_ratio': 0.6879739978331527, 'translation_length': 22860, 'reference_length': 33228}
ROUGE: {'rouge1': np.float64(0.0033333333333333335), 'rouge2': np.float64(0.0), 'rougeL': np.float64(0.0033834586466165418), 'rougeLsum': np.float64(0.0033834586466165418)}


## **Perplexity**

In [49]:
import math
import torch

def compute_perplexity_unsloth(model, tokenizer, texts, max_length):
    model.eval()
    losses = []

    for text in texts:
        enc = tokenizer(
            text,
            return_tensors="pt",
            truncation=True,
            max_length=max_length,
            add_special_tokens=True
        )

        input_ids = enc["input_ids"].to(model.device)
        labels = input_ids.clone()

        with torch.no_grad():
            outputs = model(
                input_ids=input_ids,
                labels=labels
            )

        losses.append(outputs.loss.item())

    if len(losses) == 0:
        raise ValueError("No losses collected — check eval_texts")

    mean_loss = sum(losses) / len(losses)

    return float(math.exp(mean_loss))



In [50]:
# eval_texts = raw_eval_dataset["Answers"][:50]  # small batch
# ppl = compute_perplexity(model, tokenizer, eval_texts)
# print("Perplexity:", ppl)
eval_texts = raw_eval_dataset["Answers"][:50]

ppl = compute_perplexity_unsloth(
    model,
    tokenizer,
    eval_texts,
    max_length=MAX_SEQ_LENGTH
)

print("Perplexity:", ppl, type(ppl))

Perplexity: 1.962041683699395 <class 'float'>


## **Merge LoRA → Full Model (FP16)**

In [52]:
# merged_model = FastLanguageModel.merge_and_unload(model)
merged_model = model.merge_and_unload()

/usr/local/lib/python3.12/dist-packages/peft/tuners/lora/bnb.py:397: UserWarning: Merge lora module to 4-bit linear may get different generations due to rounding errors.
  warnings.warn(


## **Save FULL Model**

In [53]:
merged_model.save_pretrained(
    "llama-3.1-8B-bangla-empathy",
    safe_serialization=True
)

tokenizer.save_pretrained("full-llama-3.1-8B-bangla-empathy")

# merged_model.save_pretrained(
#     "bengali-llama-full",
#     safe_serialization=True
# )

# tokenizer.save_pretrained("bengali-llama-full")


('full-llama-3.1-8B-bangla-empathy/tokenizer_config.json',
 'full-llama-3.1-8B-bangla-empathy/special_tokens_map.json',
 'full-llama-3.1-8B-bangla-empathy/chat_template.jinja',
 'full-llama-3.1-8B-bangla-empathy/tokenizer.json')

## ****Login to HuggingFace**

In [59]:
# from huggingface_hub import notebook_login
# notebook_login()

from huggingface_hub import login
import os
from kaggle_secrets import UserSecretsClient

try:
    # login(token=os.environ["HF_TOKEN"])
    user_secrets = UserSecretsClient()
    secret_value_0 = user_secrets.get_secret("HF_TOKEN")
    print("✓ Login successful to Hugging Face Hub")
except Exception as e:
    print(f"✗ Login failed: {e}")


✓ Login successful to Hugging Face Hub


In [64]:
from huggingface_hub import login
login()

In [65]:
import huggingface_hub
print(huggingface_hub.__version__)

0.36.0


In [66]:
from huggingface_hub import HfApi
api = HfApi()
api.whoami()


{'type': 'user',
 'id': '695b75845b1e2d9ce64fc869',
 'name': 'uzzalHossen',
 'fullname': 'Md. Uzzal Hossen',
 'isPro': False,
 'avatarUrl': 'https://cdn-avatars.huggingface.co/v1/production/uploads/noauth/cB6I07W9KZ62lfpRqMUhG.jpeg',
 'orgs': [],
 'auth': {'type': 'access_token',
  'accessToken': {'displayName': 'ForAll',
   'role': 'fineGrained',
   'createdAt': '2026-01-08T11:41:24.722Z',
   'fineGrained': {'canReadGatedRepos': True,
    'global': ['discussion.write', 'post.write'],
    'scoped': [{'entity': {'_id': '695b75845b1e2d9ce64fc869',
       'type': 'user',
       'name': 'uzzalHossen'},
      'permissions': ['repo.content.read',
       'repo.write',
       'inference.serverless.write',
       'inference.endpoints.infer.write',
       'inference.endpoints.write',
       'user.webhooks.read',
       'user.webhooks.write',
       'collection.read',
       'collection.write',
       'discussion.write',
       'job.write']}]}}}}

## **Upload to HuggingFace 🤗**

In [67]:
# merged_model.push_to_hub(
#     "uzzalHossen/full-llama-3.1-8B-bangla-empathy",
#     private=False
# )

# tokenizer.push_to_hub(
#     "uzzalHossen/full-llama-3.1-8B-bangla-empathy"
# )

merged_model.push_to_hub(
    repo_id="uzzalHossen/full-llama-3.1-8B-bangla-empathy",
    private=False
)

tokenizer.push_to_hub(
    repo_id="uzzalHossen/full-llama-3.1-8B-bangla-empathy"
)


README.md:   0%|          | 0.00/579 [00:00<?, ?B/s]

Processing Files (0 / 0): |          |  0.00B /  0.00B            

New Data Upload: |          |  0.00B /  0.00B            

Saved model to https://huggingface.co/uzzalHossen/full-llama-3.1-8B-bangla-empathy


README.md:   0%|          | 0.00/585 [00:00<?, ?B/s]

Processing Files (0 / 0): |          |  0.00B /  0.00B            

New Data Upload: |          |  0.00B /  0.00B            